In [1]:
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore') 

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [2]:
tp = []
for i in train.first_party_winner:
    apd = len(train[train['first_party_winner'] == 1])/len(train)
    if i == 1:
        tp.append(apd)
    else:
        tp.append(abs(1-apd))
train['first_party_winner'] = tp

In [3]:
train

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",0.665456
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0.334544
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,0.665456
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0.334544
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",0.665456
...,...,...,...,...,...
2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...,0.665456
2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ...",0.665456
2475,TRAIN_2475,Peguero,United States,"In 1992, the District Court sentenced Manuel D...",0.334544
2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per...",0.334544


In [4]:
train.facts[0]

'On June 27, 1962, Phil St. Amant, a candidate for public office, made a television speech in Baton Rouge, Louisiana.  During this speech, St. Amant accused his political opponent of being a Communist and of being involved in criminal activities with the head of the local Teamsters Union.  Finally, St. Amant implicated Herman Thompson, an East Baton Rouge deputy sheriff, in a scheme to move money between the Teamsters Union and St. Amant’s political opponent. \nThompson successfully sued St. Amant for defamation.  Louisiana’s First Circuit Court of Appeals reversed, holding that Thompson did not show St. Amant acted with “malice.”  Thompson then appealed to the Supreme Court of Louisiana.  That court held that, although public figures forfeit some of their First Amendment protection from defamation, St. Amant accused Thompson of a crime with utter disregard of whether the remarks were true.  Finally, that court held that the First Amendment protects uninhibited, robust debate, rather t

In [5]:
import re

reg = re.compile('[a-zA-Z]+')
mat = reg.findall(train.facts[0])
print(mat)

' '.join(mat)

['On', 'June', 'Phil', 'St', 'Amant', 'a', 'candidate', 'for', 'public', 'office', 'made', 'a', 'television', 'speech', 'in', 'Baton', 'Rouge', 'Louisiana', 'During', 'this', 'speech', 'St', 'Amant', 'accused', 'his', 'political', 'opponent', 'of', 'being', 'a', 'Communist', 'and', 'of', 'being', 'involved', 'in', 'criminal', 'activities', 'with', 'the', 'head', 'of', 'the', 'local', 'Teamsters', 'Union', 'Finally', 'St', 'Amant', 'implicated', 'Herman', 'Thompson', 'an', 'East', 'Baton', 'Rouge', 'deputy', 'sheriff', 'in', 'a', 'scheme', 'to', 'move', 'money', 'between', 'the', 'Teamsters', 'Union', 'and', 'St', 'Amant', 's', 'political', 'opponent', 'Thompson', 'successfully', 'sued', 'St', 'Amant', 'for', 'defamation', 'Louisiana', 's', 'First', 'Circuit', 'Court', 'of', 'Appeals', 'reversed', 'holding', 'that', 'Thompson', 'did', 'not', 'show', 'St', 'Amant', 'acted', 'with', 'malice', 'Thompson', 'then', 'appealed', 'to', 'the', 'Supreme', 'Court', 'of', 'Louisiana', 'That', 'cour

'On June Phil St Amant a candidate for public office made a television speech in Baton Rouge Louisiana During this speech St Amant accused his political opponent of being a Communist and of being involved in criminal activities with the head of the local Teamsters Union Finally St Amant implicated Herman Thompson an East Baton Rouge deputy sheriff in a scheme to move money between the Teamsters Union and St Amant s political opponent Thompson successfully sued St Amant for defamation Louisiana s First Circuit Court of Appeals reversed holding that Thompson did not show St Amant acted with malice Thompson then appealed to the Supreme Court of Louisiana That court held that although public figures forfeit some of their First Amendment protection from defamation St Amant accused Thompson of a crime with utter disregard of whether the remarks were true Finally that court held that the First Amendment protects uninhibited robust debate rather than an open season to shoot down the good name 

In [6]:
from nltk.corpus import stopwords, names
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
stops = stopwords.words('english')
stops.append('a')
wnl = WordNetLemmatizer()

In [7]:
reg = re.compile('[a-zA-Z]+')
for i in range(len(train)):
    first = reg.findall(train.first_party[i]) 
    second = reg.findall(train.second_party[i])
    
    tp = word_tokenize(train.facts[i])
    tp = ' '.join(tp)    
    
    fact = ' '.join(reg.findall(tp))

    for f in first:
        if f in fact:
            fact = fact.replace(f' {f} ',' <first> ')
    for f in second:
        if f in fact:
            fact = fact.replace(f' {f} ',' <second> ')
    
    tp = fact.split()
    tp = [i.lower() for i in tp]
    tp = [wnl.lemmatize(i) for i in tp]
    for j in tp:
        if j in stops:
            tp.remove(j)

    fact = ' '.join(tp)  
    tp = word_tokenize(fact)
    fact = ' '.join(tp)
                
    train.loc[i,'facts'] = fact
            
for i in range(len(test)):
    first = reg.findall(test.first_party[i])
    second = reg.findall(test.second_party[i])
    
    tp = word_tokenize(test.facts[i])
    tp = ' '.join(tp)        

    fact = ' '.join(reg.findall(tp))
    
#     for s in stops:
#         if s in fact:
#             fact = fact.replace(s,'')
    for f in first:
        if f in fact:
            fact = fact.replace(f' {f} ',' <first> ')
    for f in second:
        if f in fact:
            fact = fact.replace(f' {f} ',' <second> ')
    
    tp = fact.split()
    tp = [i.lower() for i in tp]
    tp = [wnl.lemmatize(i) for i in tp]

    for j in tp:
        if j in stops:
            tp.remove(j)
    
    fact = ' '.join(tp)
    tp = word_tokenize(fact)
    fact = ' '.join(tp)        
    
    test.loc[i,'facts'] = fact

In [8]:
test.facts[1]

'lexecon < second > wa defendant class action lawsuit usc section lawsuit wa transferred pretrial proceeding district arizona section authorizes judicial panel multidistrict litigation transfer civil action common issue fact district coordinated consolidated pretrial proceeding provides panel shall remand any such action original district or conclusion such pretrial proceeding claim dismissed < second > brought suit < first > < first > < first > < first > < first > < first > others < first > class action lawsuit northern district illinois ultimately panel section ordered the case transferred the district arizona afterwards < second > moved the arizona district court remand the case illinois < first > filed countermotion requesting the arizona district court invoke section a transfer the case itself trial ultimately the court assigned the case to itself < first > the court appeal affirmed it judgment'

In [10]:
import numpy as np 
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(lowercase=True,max_features=20000)
vectorizer.fit(train.facts)
tp = vectorizer.transform(train.facts).toarray()

for i in range(1,len(tp[0])+1):
    train[i] = [j[i-1] for j in tp]
    
train

,ID,first_party,second_party,facts,first_party_winner,1,2,3,4,5,...,13379,13380,13381,13382,13383,13384,13385,13386,13387,13388
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,june < first > < first > < first > candidate p...,0.665456,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,ramon nelson wa riding bike suffered lethal bl...,0.334544,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",alabama state court convicted < first > < firs...,0.665456,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TRAIN_0003,Linkletter,Walker,victor < first > wa convicted state court evid...,0.334544,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TRAIN_0004,William Earl Fikes,Alabama,april selma < second > intruder broke apartmen...,0.665456,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",congress amended clean air act energy policy a...,0.665456,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.",alliance < second > < second > < second > inve...,0.665456,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2475,TRAIN_2475,Peguero,United States,district court sentenced manuel < first > mont...,0.334544,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,march enrico < second > < second > lawful perm...,0.334544,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
tp = vectorizer.transform(test.facts).toarray()

for i in range(1,len(tp[0])+1):
    test[i] = [j[i-1] for j in tp]
    
test

,ID,first_party,second_party,facts,1,2,3,4,5,6,...,13379,13380,13381,13382,13383,13384,13385,13386,13387,13388
0,TEST_0000,Salerno,United States,bail reform act allowed federal court detain a...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TEST_0001,Milberg Weiss Bershad Hynes and Lerach,"Lexecon, Inc.",lexecon < second > wa defendant class action l...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,TEST_0002,No. 07-582\t Title: \t Federal Communications ...,"Fox Television Stations, Inc., et al.",< second > < second > < second > broadcast bil...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TEST_0003,Harold Kaufman,United States,trial armed robbery federally insured saving l...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TEST_0004,Berger,Hanlon,magistrate judge issued warrant authorizing se...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,TEST_1235,"Haitian Centers Council, Inc., et al.","Chris Sale, Acting Commissioner, Immigration A...",according executive order signed president geo...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1236,TEST_1236,Whitman,"American Trucking Associations, Inc.",section clean air act caa requires environment...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1237,TEST_1237,Linda A. Matteo and John J. Madigan,William G. Barr,linda < first > < first > < first > < first > ...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1238,TEST_1238,Washington State Apple Advertising Commission,Hunt,north carolina board agriculture adopted regul...,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
train.iloc[:,4:]

,first_party_winner,1,2,3,4,5,6,7,8,9,...,13379,13380,13381,13382,13383,13384,13385,13386,13387,13388
0,0.665456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.334544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.665456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.334544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.665456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,0.665456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2474,0.665456,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2475,0.334544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2476,0.334544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
test.iloc[:,4:]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,...,13139,13140,13141,13142,13143,13144,13145,13146,13147,13148,13149,13150,13151,13152,13153,13154,13155,13156,13157,13158,13159,13160,13161,13162,13163,13164,13165,13166,13167,13168,13169,13170,13171,13172,13173,13174,13175,13176,13177,13178,13179,13180,13181,13182,13183,13184,13185,13186,13187,13188,13189,13190,13191,13192,13193,13194,13195,13196,13197,13198,13199,13200,13201,13202,13203,13204,13205,13206,13207,13208,13209,13210,13211,13212,13213,13214,13215,13216,13217,13218,13219,13220,13221,13222,13223,13224,13225,13226,13227,13228,13229,13230,13231,13232,13233,13234,13235,13236,13237,13238,13239,13240,13241,13242,13243,13244,13245,13246,13247,13248,13249,13250,13251,13252,13253,13254,13255,13256,13257,13258,13259,13260,13261,13262,13263,13264,13265,13266,13267,13268,13269,13270,13271,13272,13273,13274,13275,13276,13277,13278,13279,13280,13281,13282,13283,13284,13285,13286,13287,13288,13289,13290,13291,13292,13293,13294,13295,13296,13297,13298,13299,13300,13301,13302,13303,13304,13305,13306,13307,13308,13309,13310,13311,13312,13313,13314,13315,13316,13317,13318,13319,13320,13321,13322,13323,13324,13325,13326,13327,13328,13329,13330,13331,13332,13333,13334,13335,13336,13337,13338,13339,13340,13341,13342,13343,13344,13345,13346,13347,13348,13349,13350,13351,13352,13353,13354,13355,13356,13357,13358,13359,13360,13361,13362,13363,13364,13365,13366,13367,13368,13369,13370,13371,13372,13373,13374,13375,13376,13377,13378,13379,13380,13381,13382,13383,13384,13385,13386,13387,13388
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.075923,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [13]:
from pycaret import classification
classification.setup(data=train.iloc[:,4:],
                     target='first_party_winner',
                     fold_strategy='kfold',
                     train_size=0.8,
                     session_id=77)
                     # remove_outliers=True,
                     # outliers_method='iforest')

,Description,Value
0,Session id,77
1,Target,first_party_winner
2,Target type,Binary
3,Target mapping,"0.33454397: 0, 0.665456: 1"
4,Original data shape,"(2478, 13389)"
5,Transformed data shape,"(2478, 13389)"
6,Transformed train set shape,"(1982, 13389)"
7,Transformed test set shape,"(496, 13389)"
8,Numeric features,13388
9,Preprocess,True


In [14]:
best = classification.compare_models(n_select=5,fold=10,sort='F1')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:02:55
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,SVM - Linear Kernel


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)


Processing:   0%|          | 0/73 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
test.iloc[:,4:]

In [ ]:
import seaborn as sns

tp0 = best[1].predict_proba(test.iloc[:,4:])
tp1 = best[3].predict_proba(test.iloc[:,4:])
sns.distplot([i[1]-i[0] for i in tp0],bins=100)
tp0

In [ ]:
ans = []
for i in tp0:
    if i[1]-i[0] < 0.18:
        ans.append(0)
    else:
        ans.append(1)
sum(ans)

In [ ]:
sub = pd.read_csv('./sample_submission.csv').set_index('ID')
sub['first_party_winner'] = ans
sub.to_csv('./submission.csv')